In [10]:

gcode_string = f"""G17  ; Select XY plane
G90  ; Absolute positioning
M06 T1  ; Tool change to tool 1
M03  ; Spindle on clockwise
G00 X10 Y10 Z0  ; Rapid positioning to starting point
G01 Z0 F100  ; Move down to a depth of cut at feed rate of 100 mm/min
G01 X30 Y10  ; Linear interpolation to (30,10)
G01 X50 Y30  ; Linear interpolation to (50,30)
G01 X0 Y30  ; Linear interpolation to (0,30)
G01 X10 Y10  ; Linear interpolation back to starting point
G00 Z5  ; Raise the tool above the workpiece
M05  ; Spindle stop
G28  ; Return to home position
M30  ; Program end and reset"""

In [11]:
from pygcode import Line, Word

def limit_cut_depth(gcode_string, max_depth):
    # Split the G-code string into lines
    gcode_lines = gcode_string.split('\n')
    
    # Initialize variables
    current_z = 0
    modified = False
    
    # Process each line
    for i, line_str in enumerate(gcode_lines):
        try:
            line = Line(line_str)
            
            # Check for Z movement
            z_word = next((word for word in line.block.words if word.letter == 'Z'), None)
            if z_word is not None:
                z_value = z_word.value
                # If Z value is negative (cutting) and exceeds max depth
                if z_value < 0 and abs(z_value) > max_depth:
                    # Modify the Z value
                    new_z_word = Word('Z', -max_depth)
                    new_words = [word if word.letter != 'Z' else new_z_word for word in line.block.words]
                    line.block.words = new_words
                    gcode_lines[i] = str(line)
                    modified = True
                current_z = z_value
        except ValueError:
            # Skip lines that can't be parsed (e.g., comments)
            pass
    
    # If modifications were made, return the new G-code
    if modified:
        return '\n'.join(gcode_lines)
    else:
        return gcode_string

# Example usage
max_depth = 10  # Set your desired maximum depth
modified_gcode = limit_cut_depth(gcode_string, max_depth)
print(modified_gcode)

G17  ; Select XY plane
G90  ; Absolute positioning
M06 T1  ; Tool change to tool 1
M03  ; Spindle on clockwise
G00 X10 Y10 Z0  ; Rapid positioning to starting point
G01 Z0 F100  ; Move down to a depth of cut at feed rate of 100 mm/min
G01 X30 Y10  ; Linear interpolation to (30,10)
G01 X50 Y30  ; Linear interpolation to (50,30)
G01 X0 Y30  ; Linear interpolation to (0,30)
G01 X10 Y10  ; Linear interpolation back to starting point
G00 Z5  ; Raise the tool above the workpiece
M05  ; Spindle stop
G28  ; Return to home position
M30  ; Program end and reset
